In [158]:
import pandas as pd
import sqlite3
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import csv

pd.set_option("display.max_columns", None)

In [159]:
# Data Directory
data_dir = "../../Data/Big-Data-Cup-2021"
bucketles_data_dir = '../../Data/bdc/data'
tracking_data_dir = "{}/TrackingData".format(data_dir)
db_path = '/Users/keltim01/Documents/databases/'


### Importing and cleaning the data

Thanks to [The Bucketless](https://twitter.com/the_bucketless) for the great [Tutorial](https://github.com/the-bucketless/bdc/blob/main/bdc_merge_example.ipynb) and the [Cleaned Data](https://github.com/the-bucketless/bdc/tree/main/data) as well as the [hockey_rink](https://github.com/the-bucketless/hockey_rink) package

### Play by Play Data 

In [160]:
# create and connect to database 
conn = sqlite3.connect(db_path + 'bdc_2022_test.db')
cursor = conn.cursor()

In [161]:
df_pp_info = pd.read_csv("{}/pp_info.csv".format(tracking_data_dir))


In [162]:
df_pbp = pd.read_csv("{}/pxp_womens_oly_2022_v2.csv".format(data_dir))


In [163]:
df_pbp.columns

Index(['game_date', 'season_year', 'team_name', 'opp_team_name', 'venue',
       'period', 'clock_seconds', 'situation_type', 'goals_for',
       'goals_against', 'player_name', 'event', 'event_successful', 'x_coord',
       'y_coord', 'event_type', 'player_name_2', 'x_coord_2', 'y_coord_2',
       'event_detail_1', 'event_detail_2', 'event_detail_3'],
      dtype='object')

In [164]:
df_pbp['home_team'] = df_pbp.apply(lambda x: x['team_name'] if x['venue'] == 'home' else x['opp_team_name'], axis=1)
df_pbp['away_team'] = df_pbp.apply(lambda x: x['team_name'] if x['venue'] == 'away' else x['opp_team_name'], axis=1)
df_pbp['is_shot'] = 0
df_pbp['is_goal'] = 0
df_pbp['event_id'] = df_pbp['event'].astype('category').cat.codes
df_pbp['team_id'] = df_pbp['team_name'].astype('category').cat.codes
df_pbp['player_id'] = df_pbp['player_name'].astype('category').cat.codes
df_pbp['event_successful'] = df_pbp.apply(lambda x: True if x['event_successful']=='t' else False, axis=1)
df_pbp['event_detail_2'] = df_pbp.apply(lambda x: True if x['event_detail_2']=='t' else False, axis=1)
df_pbp['event_detail_3'] = df_pbp.apply(lambda x: True if x['event_detail_3']=='t' else False, axis=1)
df_pbp[f'detail_1_code'] = df_pbp[f'event_detail_1'].astype('category').cat.codes
df_pbp.loc[df_pbp['event']=='Shot', 'is_shot'] = 1
df_pbp.loc[(df_pbp['event']=='Shot') & (df_pbp['event_successful']), 'is_goal'] = 1
df_pbp['goal_diff'] = df_pbp['goals_for'].sub(df_pbp['goals_against'])
df_pbp.loc[df_pbp['period'] == 1,'game_seconds_remaining'] = 2400 + df_pbp.loc[df_pbp['period'] == 1,'clock_seconds'] 
df_pbp.loc[df_pbp['period'] == 2,'game_seconds_remaining'] = 1200 + df_pbp.loc[df_pbp['period'] == 2,'clock_seconds']
df_pbp.loc[df_pbp['period'] == 3,'game_seconds_remaining'] = df_pbp.loc[df_pbp['period'] == 3,'clock_seconds']
# mirror home team in even-numbered periods / away team in odd-numbered periods and the opposite for Canada - Usa Game (Thanks to the Bucketless for the info)
df_pbp.loc[((df_pbp['venue'] =='home') & ~(df_pbp['period'] % 2) | (df_pbp['venue'] =='away') & (df_pbp['period'] % 2)) ^ (df_pbp['team_name'].isin(["Olympic (Women) - Canada", "Olympic (Women) - United States"]) & df_pbp['opp_team_name'].isin(["Olympic (Women) - Canada", "Olympic (Women) - United States"]) ), ['x_coord', 'x_coord_2']] = 200 - df_pbp.loc[(), ['x_coord', 'x_coord_2']]

In [165]:
df_pbp

,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,player_name,event,event_successful,x_coord,y_coord,event_type,player_name_2,x_coord_2,y_coord_2,event_detail_1,event_detail_2,event_detail_3,home_team,away_team,is_shot,is_goal,event_id,team_id,player_id,detail_1_code,goal_diff,game_seconds_remaining
0,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,1200,5 on 5,0,0,Marie-Philip Poulin,Faceoff Win,True,100.0,42,Backhand,Hannah Brandt,NaN,NaN,NaN,False,False,Olympic (Women) - United States,Olympic (Women) - Canada,0,0,1,0,60,-1,0,3600.0
1,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,1199,5 on 5,0,0,Jocelyne Larocque,Puck Recovery,True,86.0,31,NaN,NaN,NaN,NaN,NaN,False,False,Olympic (Women) - United States,Olympic (Women) - Canada,0,0,4,0,42,-1,0,3599.0
2,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,1198,5 on 5,0,0,Jocelyne Larocque,Dump In/Out,False,96.0,15,NaN,NaN,NaN,NaN,NaN,False,False,Olympic (Women) - United States,Olympic (Women) - Canada,0,0,0,0,42,-1,0,3598.0
3,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,1197,5 on 5,0,0,Jocelyne Larocque,Zone Entry,True,124.0,1,Dumped,Cayla Barnes,NaN,NaN,NaN,False,False,Olympic (Women) - United States,Olympic (Women) - Canada,0,0,7,0,42,-1,0,3597.0
4,8/2/2022,2021,Olympic (Women) - United States,Olympic (Women) - Canada,home,1,1194,5 on 5,0,0,Lee Stecklein,Puck Recovery,True,NaN,73,NaN,NaN,NaN,NaN,NaN,False,False,Olympic (Women) - United States,Olympic (Women) - Canada,0,0,4,4,53,-1,0,3594.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10604,16/2/2022,2021,Olympic (Women) - Finland,Olympic (Women) - Switzerland,home,3,14,5 on 5,0,4,Noora Tulus,Zone Entry,True,125.0,83,Carried,Shannon Sigrist,NaN,NaN,NaN,False,False,Olympic (Women) - Finland,Olympic (Women) - Switzerland,0,0,7,1,74,-1,-4,14.0
10605,16/2/2022,2021,Olympic (Women) - Finland,Olympic (Women) - Switzerland,home,3,12,5 on 5,4,0,Noora Tulus,Play,True,127.0,75,Indirect,Viivi Vainikka,179.0,76.0,NaN,False,False,Olympic (Women) - Finland,Olympic (Women) - Switzerland,0,0,3,1,74,-1,4,12.0
10606,16/2/2022,2021,Olympic (Women) - Finland,Olympic (Women) - Switzerland,home,3,10,5 on 5,4,0,Viivi Vainikka,Play,True,176.0,80,Direct,Elisa Holopainen,177.0,79.0,NaN,False,False,Olympic (Women) - Finland,Olympic (Women) - Switzerland,0,0,3,1,102,-1,4,10.0
10607,16/2/2022,2021,Olympic (Women) - Finland,Olympic (Women) - Switzerland,home,3,6,5 on 5,4,0,Elisa Holopainen,Play,True,194.0,20,Direct,Ronja Savolainen,93.0,10.0,NaN,False,False,Olympic (Women) - Finland,Olympic (Women) - Switzerland,0,0,3,1,23,-1,4,6.0


In [166]:
df_pbp.to_sql('play_by_play', conn, if_exists='replace')

### Powerplay Info 

In [167]:
df_pp_info.columns

Index(['game_name', 'penalty_number', 'start_video_clock_seconds',
       'end_video_clock_seconds', 'start_period', 'end_period',
       'start_game_clock_seconds', 'end_game_clock_seconds'],
      dtype='object')

In [168]:
df_pp_info.head()

,game_name,penalty_number,start_video_clock_seconds,end_video_clock_seconds,start_period,end_period,start_game_clock_seconds,end_game_clock_seconds
0,2022-02-08 Canada at USA,1,1278,1314,1,1,386,350
1,2022-02-08 Canada at USA,2,1624,1744,1,1,216,96
2,2022-02-08 Canada at USA,3,409,562,2,2,991,871
3,2022-02-08 Canada at USA,4,898,898,2,2,613,613
4,2022-02-08 Canada at USA,5,1690,1927,2,2,272,152


In [169]:
df_pp_info.loc[df_pp_info['start_period'] == 1,'start_game_seconds_remaining'] = 2400 + df_pp_info.loc[df_pp_info['start_period'] == 1,'start_game_clock_seconds'] 
df_pp_info.loc[df_pp_info['start_period'] == 2,'start_game_seconds_remaining'] = 1200 + df_pp_info.loc[df_pp_info['start_period'] == 2,'start_game_clock_seconds']
df_pp_info.loc[df_pp_info['start_period'] == 3,'start_game_seconds_remaining'] = df_pp_info.loc[df_pp_info['start_period'] == 3,'end_game_clock_seconds']
df_pp_info.loc[df_pp_info['end_period'] == 1,'end_game_seconds_remaining'] = 2400 + df_pp_info.loc[df_pp_info['end_period'] == 1,'end_game_clock_seconds'] 
df_pp_info.loc[df_pp_info['end_period'] == 2,'end_game_seconds_remaining'] = 1200 + df_pp_info.loc[df_pp_info['end_period'] == 2,'end_game_clock_seconds']
df_pp_info.loc[df_pp_info['end_period'] == 3,'end_game_seconds_remaining'] = df_pp_info.loc[df_pp_info['end_period'] == 3,'end_game_clock_seconds']

In [170]:
df_pp_info.to_sql('powerplay_info', conn, if_exists='replace')

### tracking data 

In [173]:
os.listdir('{}/{}'.format(bucketles_data_dir, game_name))

['2022-02-08 Canada at USA P2 PP5.csv',
 '2022-02-08 Canada at USA P3 PP7.csv',
 '2022-02-08 Canada at USA P2 PP3.csv',
 '2022-02-08 Canada at USA P3 PP6.csv',
 '2022-02-08 Canada at USA P1 PP2.csv',
 '2022-02-08 Canada at USA P1 PP1.csv']

In [177]:
bucketless_games = [x for x in os.listdir('{}'.format(bucketles_data_dir)) if x.startswith('2022')]

In [178]:
tracking_games = [x for x in os.listdir('{}'.format(tracking_data_dir)) if bool(re.search('\d{4}', x))]

In [179]:
for game in tracking_games:
    if game in bucketless_games:
        df_tracking = pd.read_csv("{}/{}/{} P1 PP1.csv".format(bucketles_data_dir, game, game))
        game_date, away_team, _, home_team = game.split(' ')
        df_tracking.loc[df_tracking['period'] == 1,'game_seconds'] = 2400 + df_tracking.loc[df_tracking['period'] == 1,'game_seconds'] 
        df_tracking.loc[df_tracking['period'] == 2,'game_seconds'] = 1200 + df_tracking.loc[df_tracking['period'] == 2,'game_seconds']
        df_tracking['venue'] = df_tracking['team_name'].map({home_team: 'home', away_team: 'away'})
        df_roster = pd.read_csv('{}/{}/{} roster.csv'.format(tracking_data_dir, game, game))
        df_roster = df_roster.rename(columns={'jn': 'jersey_number', 'team':'venue'})
        df_tracking = df_tracking.merge(df_roster, on=['venue','jersey_number'],how="left")
        df_tracking = df_tracking.rename(columns={'x_ft': "x_coord", 'y_ft': "y_coord"})
        df_pp_info.to_sql('tracking', conn, if_exists='append')
